<a href="https://colab.research.google.com/github/jmsato/CS4650_HW5/blob/main/Main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [80]:
import spacy
import en_core_web_lg
from newsapi.newsapi_client import NewsApiClient
import pickle
import pandas as pd
from datetime import datetime
from collections import Counter
from google.colab import drive

#Function to extract part-of-speech tagging
pos_tag = {'VERB', 'NOUN', 'PROPN'}
def get_keywords_eng(text):
  result = []
  for token in nlp_eng(text):
    if (token.text in nlp_eng.Defaults.stop_words):
      continue
    if (token.pos_ in pos_tag):
      result.append(token.text)
  return result

#API request
nlp_eng = en_core_web_lg.load()
newsapi = NewsApiClient(api_key='eb18bdd4d4e645978ac8143372f2184a')
temp = newsapi.get_everything(q='coronavirus', language='en', from_param='2021-02-19', to='2021-03-18', sort_by='relevancy')
articles = temp['articles']

#Save result to Google Drive
filename = 'articlesCOVID.pckl'
pickle.dump(articles, open(filename, 'wb'))
loaded_model = pickle.load(open(filename, 'rb'))
filepath = 'articlesCOVID.pckl'
pickle.dump(loaded_model, open(filepath, 'wb'))

#Get only title, date, description, and content of articles
dados = list()
for i, article in enumerate(articles):
  title = article['title']
  date = article['publishedAt']
  date = datetime.fromisoformat(date[:-1]).date()
  description = article['description']
  content = article['content']
  dados.append({'title': title, 'date': date, 'desc': description, 'content': content, 'keywords': [None] * 5})
df = pd.DataFrame(dados)

#Clean the data
df = df.dropna()

#Extract parts of speech data from content
for content in df.content.values:
  results = list()
  results.append([('#' + x[0]) for x in Counter(get_keywords_eng(content)).most_common(5)])
  df.at[df.index[df.content == content][0], 'keywords'] = results[0]

#Print example of data and save into a csv file to save onto Google Drive
df.head()
print(df)
drive.mount('drive')
df.to_csv('CovidArticles.csv')
!cp CovidArticles.csv "drive/My Drive/"

Int64Index([0], dtype='int64')
Int64Index([1], dtype='int64')
Int64Index([2], dtype='int64')
Int64Index([3], dtype='int64')
Int64Index([4], dtype='int64')
Int64Index([5], dtype='int64')
Int64Index([6], dtype='int64')
Int64Index([7], dtype='int64')
Int64Index([8], dtype='int64')
Int64Index([9], dtype='int64')
Int64Index([10], dtype='int64')
Int64Index([11], dtype='int64')
Int64Index([12], dtype='int64')
Int64Index([13], dtype='int64')
Int64Index([14], dtype='int64')
Int64Index([15], dtype='int64')
Int64Index([16], dtype='int64')
Int64Index([18], dtype='int64')
Int64Index([19], dtype='int64')
                                                title  ...                                           keywords
0           Coronavirus Briefing: What Happened Today  ...            [#going, #think, #months, #look, #U.S.]
1         Your N.B.A. Coronavirus Questions, Answered  ...             [#Knight, #seen, #Wayne, #ads, #Space]
2           Coronavirus Briefing: What Happened Today  ...  [#ethicists,